In [ ]:
# =============================================================================
# ⚙️ CONFIGURATION CENTRALISÉE VISUALISATION
# =============================================================================

import sys
import os
import json
from pathlib import Path

def detect_environment():
    """Détecte automatiquement si on est sur Colab ou en local"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

def setup_colab_environment(video_name):
    """Configure l'environnement Colab en récupérant le backup"""
    from google.colab import drive
    import zipfile
    import shutil
    
    print("🔬 Mode Colab détecté")
    print("📦 Récupération du backup depuis Google Drive...")
    
    # Monter Google Drive
    print("   📁 Montage Google Drive...")
    drive.mount('/content/drive')
    
    # Chemin du ZIP de backup
    zip_path = f'/content/drive/MyDrive/{video_name}.zip'
    extraction_path = '/content/videos'
    
    if not Path(zip_path).exists():
        raise FileNotFoundError(f"❌ Backup non trouvé: {zip_path}")
    
    print(f"   📦 Extraction du backup...")
    print(f"      Source: {zip_path}")
    print(f"      Destination: {extraction_path}")
    
    # Nettoyer et créer le dossier de destination
    if Path(extraction_path).exists():
        shutil.rmtree(extraction_path)
    os.makedirs(extraction_path, exist_ok=True)
    
    # Extraire le ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)
    
    # Vérification
    file_count = sum([len(files) for r, d, files in os.walk(extraction_path)])
    print(f"   ✅ {file_count} fichiers restaurés")
    
    return extraction_path

def setup_local_environment():
    """Configure l'environnement local en récupérant cfg existant"""
    print("🖥️ Mode Local détecté")
    
    # Vérifier si cfg existe déjà (depuis SAM_inference_refactor.ipynb)
    if 'cfg' in globals():
        print("   ✅ Configuration cfg trouvée depuis SAM_inference_refactor.ipynb")
        return globals()['cfg']
    else:
        print("   ⚠️ Configuration cfg non trouvée")
        print("   💡 Assurez-vous d'avoir exécuté SAM_inference_refactor.ipynb d'abord")
        print("   🔄 Création d'une configuration basique...")
        
        # Configuration de base si cfg n'existe pas
        from pathlib import Path
        class BasicConfig:
            def __init__(self, video_name):
                self.VIDEO_NAME = video_name
                self.videos_dir = Path("../data/videos")
                self.output_dir = self.videos_dir / "outputs" / video_name
                self.frames_dir = self.output_dir / "frames" 
                self.output_json_path = self.output_dir / f"{video_name}_project.json"
                
        return BasicConfig("SD_13_06_2025_1_PdB_S1_T959s")  # Valeur par défaut

class VizConfig:
    """Configuration centralisée pour la visualisation"""
    
    def __init__(self, video_name=None):
        # Détection automatique de l'environnement
        self.USING_COLAB = detect_environment()
        
        # Configuration selon l'environnement
        if self.USING_COLAB:
            # Mode Colab : récupération du backup
            if video_name is None:
                video_name = "SD_13_06_2025_1_PdB_S1_T959s"  # Par défaut
            
            extraction_path = setup_colab_environment(video_name)
            self._setup_colab_paths(video_name, extraction_path)
            
        else:
            # Mode Local : récupération de cfg existant
            existing_cfg = setup_local_environment()
            self._setup_local_paths(existing_cfg)
    
    def _setup_colab_paths(self, video_name, extraction_path):
        """Configure les chemins pour Colab"""
        self.VIDEO_NAME = video_name
        self.videos_dir = Path(extraction_path)
        self.output_dir = self.videos_dir / "outputs" / video_name
        self.frames_dir = self.output_dir / "frames"
        self.masks_dir = self.output_dir / "masks"
        self.output_json_path = self.output_dir / f"{video_name}_project.json"
        self.video_path = self.videos_dir / f"{video_name}.mp4"
        self.output_video_path = self.output_dir / f"{video_name}_annotated.mp4"
    
    def _setup_local_paths(self, existing_cfg):
        """Configure les chemins pour Local depuis cfg existant"""
        # Reprendre les chemins de cfg si disponible
        if hasattr(existing_cfg, 'VIDEO_NAME'):
            self.VIDEO_NAME = existing_cfg.VIDEO_NAME
            self.videos_dir = existing_cfg.videos_dir
            self.output_dir = existing_cfg.output_dir
            self.frames_dir = existing_cfg.frames_dir
            self.masks_dir = existing_cfg.masks_dir if hasattr(existing_cfg, 'masks_dir') else self.output_dir / "masks"
            self.output_json_path = existing_cfg.output_json_path
            self.video_path = existing_cfg.video_path if hasattr(existing_cfg, 'video_path') else self.videos_dir / f"{self.VIDEO_NAME}.mp4"
            self.output_video_path = existing_cfg.output_video_path if hasattr(existing_cfg, 'output_video_path') else self.output_dir / f"{self.VIDEO_NAME}_annotated.mp4"
        else:
            # Configuration de base
            self.VIDEO_NAME = existing_cfg.VIDEO_NAME if hasattr(existing_cfg, 'VIDEO_NAME') else "SD_13_06_2025_1_PdB_S1_T959s"
            self.videos_dir = existing_cfg.videos_dir if hasattr(existing_cfg, 'videos_dir') else Path("../data/videos")
            self.output_dir = self.videos_dir / "outputs" / self.VIDEO_NAME
            self.frames_dir = self.output_dir / "frames"
            self.masks_dir = self.output_dir / "masks"
            self.output_json_path = self.output_dir / f"{self.VIDEO_NAME}_project.json"
            self.video_path = self.videos_dir / f"{self.VIDEO_NAME}.mp4"
            self.output_video_path = self.output_dir / f"{self.VIDEO_NAME}_annotated.mp4"
    
    def validate_files(self):
        """Valide que les fichiers nécessaires existent"""
        if not self.output_json_path.exists():
            raise FileNotFoundError(f"❌ Fichier projet non trouvé: {self.output_json_path}")
        if not self.frames_dir.exists() or not list(self.frames_dir.glob("*.jpg")):
            raise FileNotFoundError(f"❌ Frames non trouvées dans: {self.frames_dir}")
        
        print("✅ Tous les fichiers nécessaires sont présents")
        return True
    
    def display_config(self):
        """Affiche la configuration actuelle"""
        print(f"📋 CONFIGURATION VISUALISATION:")
        print(f"   🌍 Environnement: {'🔬 Colab' if self.USING_COLAB else '🖥️ Local'}")
        print(f"   🎬 Vidéo: {self.VIDEO_NAME}")
        print(f"   📁 Dossier vidéos: {self.videos_dir}")
        print(f"   📄 Fichier projet: {self.output_json_path}")
        print(f"   🖼️  Frames: {self.frames_dir}")
        print(f"   🎥 Vidéo annotée: {self.output_video_path}")

# =============================================================================
# 🚀 INITIALISATION AUTOMATIQUE
# =============================================================================

# Initialisation automatique selon l'environnement
# Si vous voulez un nom de vidéo différent, modifiez ici :
VIDEO_NAME_OVERRIDE = None  # Ou "VotreNomDeVideo" pour forcer

viz_cfg = VizConfig(VIDEO_NAME_OVERRIDE)
viz_cfg.display_config()

# Validation des fichiers
try:
    viz_cfg.validate_files()
    print(f"\n🎉 Configuration visualisation prête!")
except FileNotFoundError as e:
    print(f"\n❌ {e}")
    if viz_cfg.USING_COLAB:
        print("💡 Assurez-vous que le backup est bien présent sur Google Drive")
    else:
        print("💡 Exécutez d'abord SAM_inference_refactor.ipynb pour générer les données")

In [17]:
import os
import json
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any
import cv2
from tqdm import tqdm


@dataclass
class VideoInfo:
    """Classe pour stocker les informations vidéo"""
    fps: float
    duration: float
    frame_count: int
    width: int
    height: int

    @property
    def aspect_ratio(self) -> float:
        return self.width / self.height


class VideoPaths:
    """Gestionnaire des chemins de fichiers - Compatible avec SAM_inference.ipynb"""

    def __init__(self, video_name: str, videos_dir: str = "./videos"):
        self.video_name = video_name
        self.videos_dir = Path(videos_dir)

        # Structure des chemins identique à SAM_inference.ipynb
        self.video_path = self.videos_dir / f"{video_name}.mp4"
        self.config_path = self.videos_dir / f"{video_name}_config.json"
        self.output_dir = self.videos_dir / "outputs" / video_name
        self.frames_dir = self.output_dir / "frames"
        self.masks_dir = self.output_dir / "masks"
        self.output_video_path = self.output_dir / f"{video_name}_annotated.mp4"
        self.json_output = self.output_dir / f"{video_name}_project.json"

        # Création des répertoires si nécessaire
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.frames_dir.mkdir(exist_ok=True)
        self.masks_dir.mkdir(exist_ok=True)

    @property
    def raw_images_dir(self) -> str:
        """Compatibilité: retourne le chemin des frames"""
        return str(self.frames_dir)

    @property
    def mask_images_dir(self) -> str:
        """Compatibilité: retourne le chemin des masques"""
        return str(self.masks_dir)

    @property
    def video(self) -> str:
        """Compatibilité: retourne le chemin de la vidéo"""
        return str(self.video_path)

    @property
    def output_video(self) -> str:
        """Compatibilité: retourne le chemin de la vidéo de sortie"""
        return str(self.output_video_path)


class BaseDrawer:
    """Classe de base pour tous les dessinateurs"""

    @staticmethod
    def get_object_color(object_info: Dict) -> str:
        """Récupère la couleur appropriée pour un objet"""
        object_type = object_info.get('type', 'unknown')

        if object_type == 'player':
            return object_info.get('jersey_color') or object_info.get('display_color', 'red')
        else:
            return object_info.get('display_color', 'blue')

    @staticmethod
    def should_display_id(object_info: Dict, object_type_filter: List[str] = None) -> bool:
        """Détermine si l'ID doit être affiché selon le type d'objet"""
        object_type = object_info.get('type', 'unknown')
        if object_type_filter is None:
            return object_type not in ['ballon', 'ball']
        return object_type in object_type_filter


class ImageDrawer(BaseDrawer):
    """Gestionnaire du dessin des objets sur l'image"""

    def __init__(self):
        self.drawing_functions = {
            'player': self._draw_player,
            'ballon': self._draw_ball,
            'ball': self._draw_ball,
            'arbitre': self._draw_referee,
            'referee': self._draw_referee,
            'staff': self._draw_staff,
            'unknown': self._draw_unknown
        }

    def _draw_player(self, ax, x: float, y: float, color: str = 'red'):
        """Dessine les ellipses pour un joueur"""
        ellipse_width, ellipse_height = 50, 20
        gap_size = 120
        arc_linewidth = 2

        for gap_center in [90, 270]:  # Haut et bas
            other_gap = 270 if gap_center == 90 else 90
            gap_start = gap_center - gap_size // 2
            gap_end = gap_center + gap_size // 2
            other_start = other_gap - gap_size // 2
            other_end = other_gap + gap_size // 2

            arc = patches.Arc((x, y-5), width=ellipse_width, height=ellipse_height,
                            angle=0, theta1=gap_end, theta2=other_start + (360 if other_start < gap_end else 0),
                            color=color, linewidth=arc_linewidth)
            ax.add_patch(arc)

    def _draw_ball(self, ax, x: float, y: float, **kwargs):
        """Dessine un triangle vert pour le ballon"""
        triangle_size = 15
        triangle = patches.Polygon(
            [(x, y-20), (x-triangle_size, y-35), (x+triangle_size, y-35)],
            closed=True, facecolor='yellow', edgecolor='yellow', linewidth=2
        )
        ax.add_patch(triangle)

    def _draw_referee(self, ax, x: float, y: float, color: str = 'black'):
        """Dessine un losange pour l'arbitre"""
        diamond_size = 12
        diamond = patches.Polygon(
            [(x, y-5-diamond_size), (x+diamond_size, y-5),
             (x, y-5+diamond_size), (x-diamond_size, y-5)],
            closed=True, facecolor=color, edgecolor='white', linewidth=2
        )
        ax.add_patch(diamond)

    def _draw_staff(self, ax, x: float, y: float, color: str = 'purple'):
        """Dessine un carré pour le staff"""
        square_size = 10
        square = patches.Rectangle(
            (x-square_size, y-5-square_size), square_size*2, square_size*2,
            facecolor=color, edgecolor='white', linewidth=2
        )
        ax.add_patch(square)

    def _draw_unknown(self, ax, x: float, y: float, color: str = 'blue'):
        """Dessine un cercle pour les objets de type inconnu"""
        circle = patches.Circle((x, y-5), radius=10,
                              facecolor=color, edgecolor='black', linewidth=2)
        ax.add_patch(circle)

    def draw_object(self, ax, x: float, y: float, object_info: Dict, object_id: str):
        """Dessine un objet sur l'image"""
        object_type = object_info.get('type', 'unknown')
        color = self.get_object_color(object_info)

        draw_function = self.drawing_functions.get(object_type, self._draw_unknown)

        # Appeler la fonction avec les bons paramètres
        if object_type in ['ballon', 'ball']:
            draw_function(ax, x, y)
        else:
            draw_function(ax, x, y, color)

        # Afficher l'ID si nécessaire
        if self.should_display_id(object_info):
            self._draw_annotation(ax, x, y, object_id, color)

    def _draw_annotation(self, ax, x: float, y: float, object_id: str, color: str):
        """Dessine l'annotation (ID) pour un objet"""
        ax.annotate(f'{object_id}', (x, y+5), xytext=(0, 0),
                   textcoords='offset points', fontsize=10, ha='center', va='center',
                   color='white', weight='bold',
                   bbox=dict(boxstyle="round,pad=0.2,rounding_size=0.5",
                           facecolor=color, alpha=0.7, edgecolor=color))


class FieldDrawer(BaseDrawer):
    """Gestionnaire du dessin des objets sur le terrain"""

    def __init__(self):
        self.drawing_functions = {
            'player': self._draw_player,
            'ball': self._draw_ball,
            'ballon': self._draw_ball,
            'referee': self._draw_referee,
            'arbitre': self._draw_referee,
            'staff': self._draw_staff,
            'unknown': self._draw_unknown
        }

    def _draw_player(self, ax, x: float, y: float, object_info: Dict, point_size: int = 70):
        """Dessine un cercle pour un joueur sur le terrain"""
        color = self.get_object_color(object_info)
        ax.scatter(x, y, color=color, s=point_size, zorder=5,
                  edgecolors='white', linewidth=1)

    def _draw_ball(self, ax, x: float, y: float, point_size: int = 70, **kwargs):
        """Dessine un triangle vert pour le ballon sur le terrain"""
        triangle_size = np.sqrt(point_size) * 0.1
        triangle_points = np.array([
            [x, y - triangle_size],
            [x - triangle_size, y + triangle_size/2],
            [x + triangle_size, y + triangle_size/2]
        ])
        triangle = patches.Polygon(triangle_points, closed=True,
                                 facecolor='yellow', edgecolor='yellow', linewidth=2, zorder=5)
        ax.add_patch(triangle)

    def _draw_referee(self, ax, x: float, y: float, point_size: int = 70, **kwargs):
        """Dessine un cercle noir pour l'arbitre sur le terrain"""
        ax.scatter(x, y, color='black', s=point_size, zorder=5,
                  edgecolors='white', linewidth=1)

    def _draw_staff(self, ax, x: float, y: float, point_size: int = 70, **kwargs):
        """Dessine un carré pour le staff sur le terrain"""
        square_size = np.sqrt(point_size) * 0.6
        square = patches.Rectangle((x - square_size/2, y - square_size/2),
                                 square_size, square_size,
                                 facecolor='purple', edgecolor='white', linewidth=2, zorder=5)
        ax.add_patch(square)

    def _draw_unknown(self, ax, x: float, y: float, point_size: int = 70, **kwargs):
        """Dessine un cercle gris pour les objets de type inconnu"""
        ax.scatter(x, y, color='gray', s=point_size, zorder=5,
                  edgecolors='white', linewidth=1)

    def draw_object(self, ax, x: float, y: float, object_info: Dict, object_id: str, point_size: int = 70):
        """Dessine un objet sur le terrain"""
        object_type = object_info.get('type', 'unknown')
        draw_function = self.drawing_functions.get(object_type, self._draw_unknown)

        # Appeler la fonction avec les bons paramètres
        if object_type in ['ball', 'ballon']:
            draw_function(ax, x, y, point_size)
        elif object_type == 'player':
            draw_function(ax, x, y, object_info, point_size)
        else:
            draw_function(ax, x, y, point_size)

        # Afficher l'ID seulement pour les joueurs
        if self.should_display_id(object_info, ['player']):
            ax.text(x, y, str(object_id), color='white', fontsize=9,
                   ha='center', va='center', weight='bold', zorder=6)


class FootballField2D:
    """Classe pour dessiner un terrain de football en 2D"""

    def __init__(self, field_length=105, field_width=68, line_color='white',
                 background_color='#2d5a27', line_width=2):
        self.field_length = field_length
        self.field_width = field_width
        self.line_color = line_color
        self.background_color = background_color
        self.line_width = line_width

        # Dimensions standards
        self.penalty_length = 16.5
        self.penalty_width = 40.32
        self.goal_area_length = 5.5
        self.goal_area_width = 18.32
        self.center_circle_radius = 9.15
        self.penalty_spot_distance = 11
        self.goal_width = 7.32
        self.goal_depth = 1.5

        # Paramètres de transformation
        self.rotation = 0
        self.half_field = None
        self.invert_x = False
        self.invert_y = False

    def _rotate_coordinates(self, coords, rotation):
        """Applique une rotation aux coordonnées"""
        if rotation == 0:
            return coords
        elif rotation == 90:
            return np.column_stack([-coords[:, 1], coords[:, 0]])
        elif rotation == 180:
            return np.column_stack([-coords[:, 0], -coords[:, 1]])
        elif rotation == 270:
            return np.column_stack([coords[:, 1], -coords[:, 0]])
        else:
            raise ValueError("La rotation doit être 0, 90, 180 ou 270 degrés")

    def draw(self, ax=None, show_plot=True, figsize=(12, 8), rotation=0,
             half_field=None, invert_x=False, invert_y=True):
        """Dessine le terrain de football"""
        # Stocker les paramètres
        self.rotation = rotation
        self.half_field = half_field
        self.invert_x = invert_x
        self.invert_y = invert_y

        if ax is None:
            fig, ax = plt.subplots(figsize=figsize)

        ax.set_facecolor(self.background_color)

        # Dessiner tous les éléments
        self._draw_field_outline(ax)
        self._draw_center_line(ax)
        self._draw_center_circle(ax)
        self._draw_penalty_areas(ax)
        self._draw_goal_areas(ax)
        self._draw_penalty_spots(ax)
        self._draw_penalty_arcs(ax)
        self._draw_goals(ax)

        # Configuration des axes
        self._configure_axes(ax, rotation, half_field, invert_x, invert_y)

        if show_plot:
            plt.tight_layout()
            plt.show()

        return ax

    def _draw_field_outline(self, ax):
        """Dessine le contour du terrain principal"""
        field_corners = np.array([
            [-self.field_length/2, -self.field_width/2],
            [self.field_length/2, -self.field_width/2],
            [self.field_length/2, self.field_width/2],
            [-self.field_length/2, self.field_width/2],
            [-self.field_length/2, -self.field_width/2]
        ])
        rotated_corners = self._rotate_coordinates(field_corners, self.rotation)
        ax.plot(rotated_corners[:, 0], rotated_corners[:, 1],
                color=self.line_color, linewidth=self.line_width)

    def _draw_center_line(self, ax):
        """Dessine la ligne médiane"""
        center_line = np.array([
            [0, -self.field_width/2],
            [0, self.field_width/2]
        ])
        rotated_line = self._rotate_coordinates(center_line, self.rotation)
        ax.plot(rotated_line[:, 0], rotated_line[:, 1],
                color=self.line_color, linewidth=self.line_width)

    def _draw_center_circle(self, ax):
        """Dessine le cercle central et le point central"""
        circle_points = 100
        theta = np.linspace(0, 2*np.pi, circle_points)
        x_circle = self.center_circle_radius * np.cos(theta)
        y_circle = self.center_circle_radius * np.sin(theta)

        circle_coords = np.column_stack([x_circle, y_circle])
        rotated_circle = self._rotate_coordinates(circle_coords, self.rotation)
        ax.plot(rotated_circle[:, 0], rotated_circle[:, 1],
                color=self.line_color, linewidth=self.line_width)

        # Point central
        center_point = np.array([[0, 0]])
        rotated_center = self._rotate_coordinates(center_point, self.rotation)
        ax.plot(rotated_center[0, 0], rotated_center[0, 1], 'o',
                color=self.line_color, markersize=3)

    # ... [Autres méthodes de dessin du terrain] ...
    def _draw_penalty_areas(self, ax):
        """Dessine les surfaces de réparation"""
        # Surface de réparation gauche
        penalty_left = np.array([
            [-self.field_length/2, -self.penalty_width/2],
            [-self.field_length/2 + self.penalty_length, -self.penalty_width/2],
            [-self.field_length/2 + self.penalty_length, self.penalty_width/2],
            [-self.field_length/2, self.penalty_width/2],
            [-self.field_length/2, -self.penalty_width/2]
        ])
        rotated_left = self._rotate_coordinates(penalty_left, self.rotation)
        ax.plot(rotated_left[:, 0], rotated_left[:, 1],
                color=self.line_color, linewidth=self.line_width)

        # Surface de réparation droite
        penalty_right = np.array([
            [self.field_length/2, -self.penalty_width/2],
            [self.field_length/2 - self.penalty_length, -self.penalty_width/2],
            [self.field_length/2 - self.penalty_length, self.penalty_width/2],
            [self.field_length/2, self.penalty_width/2],
            [self.field_length/2, -self.penalty_width/2]
        ])
        rotated_right = self._rotate_coordinates(penalty_right, self.rotation)
        ax.plot(rotated_right[:, 0], rotated_right[:, 1],
                color=self.line_color, linewidth=self.line_width)

    def _draw_goal_areas(self, ax):
        """Dessine les surfaces de but"""
        # Surface de but gauche
        goal_left = np.array([
            [-self.field_length/2, -self.goal_area_width/2],
            [-self.field_length/2 + self.goal_area_length, -self.goal_area_width/2],
            [-self.field_length/2 + self.goal_area_length, self.goal_area_width/2],
            [-self.field_length/2, self.goal_area_width/2],
            [-self.field_length/2, -self.goal_area_width/2]
        ])
        rotated_left = self._rotate_coordinates(goal_left, self.rotation)
        ax.plot(rotated_left[:, 0], rotated_left[:, 1],
                color=self.line_color, linewidth=self.line_width)

        # Surface de but droite
        goal_right = np.array([
            [self.field_length/2, -self.goal_area_width/2],
            [self.field_length/2 - self.goal_area_length, -self.goal_area_width/2],
            [self.field_length/2 - self.goal_area_length, self.goal_area_width/2],
            [self.field_length/2, self.goal_area_width/2],
            [self.field_length/2, -self.goal_area_width/2]
        ])
        rotated_right = self._rotate_coordinates(goal_right, self.rotation)
        ax.plot(rotated_right[:, 0], rotated_right[:, 1],
                color=self.line_color, linewidth=self.line_width)

    def _draw_penalty_spots(self, ax):
        """Dessine les points de penalty"""
        penalty_spots = np.array([
            [-self.field_length/2 + self.penalty_spot_distance, 0],
            [self.field_length/2 - self.penalty_spot_distance, 0]
        ])
        rotated_spots = self._rotate_coordinates(penalty_spots, self.rotation)
        ax.plot(rotated_spots[:, 0], rotated_spots[:, 1], 'o',
                color=self.line_color, markersize=3)

    def _draw_penalty_arcs(self, ax):
        """Dessine les arcs de cercle des surfaces de réparation"""
        arc_angles = np.linspace(-np.pi/3, np.pi/3, 50)

        # Arc gauche
        arc_x_left = (-self.field_length/2 + self.penalty_spot_distance +
                      self.center_circle_radius * np.cos(arc_angles))
        arc_y_left = self.center_circle_radius * np.sin(arc_angles)

        mask_left = arc_x_left >= -self.field_length/2 + self.penalty_length
        if np.any(mask_left):
            arc_left_coords = np.column_stack([arc_x_left[mask_left], arc_y_left[mask_left]])
            rotated_arc_left = self._rotate_coordinates(arc_left_coords, self.rotation)
            ax.plot(rotated_arc_left[:, 0], rotated_arc_left[:, 1],
                    color=self.line_color, linewidth=self.line_width)

        # Arc droit
        arc_x_right = (self.field_length/2 - self.penalty_spot_distance +
                       self.center_circle_radius * np.cos(np.pi - arc_angles))
        arc_y_right = self.center_circle_radius * np.sin(np.pi - arc_angles)

        mask_right = arc_x_right <= self.field_length/2 - self.penalty_length
        if np.any(mask_right):
            arc_right_coords = np.column_stack([arc_x_right[mask_right], arc_y_right[mask_right]])
            rotated_arc_right = self._rotate_coordinates(arc_right_coords, self.rotation)
            ax.plot(rotated_arc_right[:, 0], rotated_arc_right[:, 1],
                    color=self.line_color, linewidth=self.line_width)

    def _draw_goals(self, ax):
        """Dessine les buts"""
        # But gauche
        goal_left_posts = np.array([
            [-self.field_length/2, -self.goal_width/2],
            [-self.field_length/2 - self.goal_depth, -self.goal_width/2],
            [-self.field_length/2 - self.goal_depth, self.goal_width/2],
            [-self.field_length/2, self.goal_width/2]
        ])
        rotated_left_goal = self._rotate_coordinates(goal_left_posts, self.rotation)
        ax.plot(rotated_left_goal[:, 0], rotated_left_goal[:, 1],
                color=self.line_color, linewidth=self.line_width + 1)

        # But droit
        goal_right_posts = np.array([
            [self.field_length/2, -self.goal_width/2],
            [self.field_length/2 + self.goal_depth, -self.goal_width/2],
            [self.field_length/2 + self.goal_depth, self.goal_width/2],
            [self.field_length/2, self.goal_width/2]
        ])
        rotated_right_goal = self._rotate_coordinates(goal_right_posts, self.rotation)
        ax.plot(rotated_right_goal[:, 0], rotated_right_goal[:, 1],
                color=self.line_color, linewidth=self.line_width + 1)

    def _configure_axes(self, ax, rotation, half_field, invert_x, invert_y):
        """Configure les axes et l'apparence générale"""
        ax.set_aspect('equal')

        # Calculer les limites
        if rotation in [0, 180]:
            max_x = self.field_length/2 + 10
            max_y = self.field_width/2 + 10
        else:
            max_x = self.field_width/2 + 10
            max_y = self.field_length/2 + 10

        x_min, x_max = -max_x, max_x
        y_min, y_max = -max_y, max_y

        # Ajuster pour demi-terrain
        if half_field == 'left':
            if rotation == 0:
                x_max = 5
            elif rotation == 90:
                y_max = 5
            elif rotation == 180:
                x_min = -5
            elif rotation == 270:
                y_min = -5
        elif half_field == 'right':
            if rotation == 0:
                x_min = -5
            elif rotation == 90:
                y_min = -5
            elif rotation == 180:
                x_max = 5
            elif rotation == 270:
                y_max = 5

        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])

        if invert_x:
            ax.invert_xaxis()
        if invert_y:
            ax.invert_yaxis()

        # Style
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])


class DataExtractor:
    """Gestionnaire d'extraction des données depuis les annotations"""

    @staticmethod
    def extract_image_points(frame_annotations: List, objects_config: Dict) -> Tuple[List, List, List, List]:
        """Extrait les coordonnées image avec les informations des objets"""
        x_coords, y_coords, object_ids, object_infos = [], [], [], []

        for annotation in frame_annotations:
            if not annotation or 'points' not in annotation:
                continue

            output = annotation['points'].get('output')
            if not output:  # Vérifier que output n'est pas None ou vide
                continue

            image_points = output.get('image', {})
            center_bottom = image_points.get('CENTER_BOTTOM')

            if center_bottom and 'x' in center_bottom and 'y' in center_bottom:
                x_coords.append(center_bottom['x'])
                y_coords.append(center_bottom['y'])

                object_id = annotation['objectId']
                object_ids.append(object_id)
                object_infos.append(objects_config.get(str(object_id), {}))

        return x_coords, y_coords, object_ids, object_infos

    @staticmethod
    def extract_field_points(frame_annotations: List, objects_config: Dict) -> Dict:
        """Extrait les coordonnées terrain avec les informations des objets"""
        field_points = {}

        for annotation in frame_annotations:
            if not annotation or 'points' not in annotation:
                continue

            output = annotation['points'].get('output')
            if not output:  # Vérifier que output n'est pas None ou vide
                continue

            field_points_data = output.get('field', {})
            center_bottom = field_points_data.get('CENTER_BOTTOM')

            if center_bottom and isinstance(center_bottom, dict):
                object_id = annotation['objectId']
                field_points[object_id] = {
                    'x': center_bottom['x'],
                    'y': center_bottom['y'],
                    'info': objects_config.get(str(object_id), {})
                }

        return field_points


class FootballVisualizer:
    """Classe principale pour la visualisation des objets football"""

    def __init__(self, viz_cfg):  # ← CHANGEMENT : prend viz_cfg au lieu de video_name
        self.cfg = viz_cfg  # ← CHANGEMENT : stocke la config centralisée
        self.image_drawer = ImageDrawer()
        self.field_drawer = FieldDrawer()
        self.data_extractor = DataExtractor()

        # Configuration minimap par défaut
        self.minimap_config = {
            'rotation': 90,
            'half_field': 'right',
            'invert_x': False,
            'invert_y': True,
            'transparency': 0.65,
            'size': "35%",
            'position': 'lower center'
        }


    def load_project_data(self) -> Dict:
        """Charge les données du projet depuis le fichier JSON"""
        with open(self.cfg.output_json_path) as f:  # ← CHANGEMENT : utilise cfg
            return json.load(f)

    def load_frame_image(self, frame_id: str) -> Image.Image:
        """Charge l'image d'une frame"""
        # CHANGEMENT : utilise cfg au lieu de paths
        image_path_jpg = self.cfg.frames_dir / f"{int(frame_id):05d}.jpg"
        image_path_jpeg = self.cfg.frames_dir / f"{int(frame_id):05d}.jpeg"

        if image_path_jpg.exists():
            return Image.open(image_path_jpg)
        elif image_path_jpeg.exists():
            return Image.open(image_path_jpeg)
        else:
            print(f"⚠️ Image non trouvée: {image_path_jpg} ni {image_path_jpeg}")
            return Image.new('RGB', (1920, 1080), color='black')

    def configure_minimap(self, **config):
        """Configure les paramètres de la minimap"""
        self.minimap_config.update(config)

    def visualize_frame(self, frame_id: str, show_minimap: bool = True, figsize: Tuple[int, int] = (15, 8)):
        """Visualise une frame complète avec image et minimap optionnelle"""
        # Charger les données
        project = self.load_project_data()
        objects_config = project.get('objects', {})
        frame_annotations = project['annotations'][frame_id]
        img = self.load_frame_image(frame_id)

        # Extraire les données
        x_coords, y_coords, object_ids, object_infos = self.data_extractor.extract_image_points(
            frame_annotations, objects_config)
        field_points = self.data_extractor.extract_field_points(frame_annotations, objects_config)

        # Créer la figure principale
        fig, ax_main = plt.subplots(figsize=figsize)
        ax_main.imshow(img)
        ax_main.axis('off')

        # Dessiner les objets sur l'image
        self._draw_image_objects(ax_main, x_coords, y_coords, object_ids, object_infos)

        # Ajouter la minimap si demandée et si des points terrain sont disponibles
        if show_minimap and field_points:
            self._create_minimap(ax_main, field_points, frame_id, len(frame_annotations))
        elif show_minimap:
            print("🟡 Pas de coordonnées terrain disponibles - minimap désactivée")

        plt.tight_layout()
        plt.show()

        return fig, ax_main

    def _draw_image_objects(self, ax, x_coords: List, y_coords: List,
                          object_ids: List, object_infos: List):
        """Dessine tous les objets sur l'image"""
        for x, y, obj_id, obj_info in zip(x_coords, y_coords, object_ids, object_infos):
            self.image_drawer.draw_object(ax, x, y, obj_info, obj_id)

    def _create_minimap(self, ax_main, field_points: Dict, frame_id: str, total_objects: int):
        """Crée la minimap avec le terrain et les objets"""
        config = self.minimap_config

        # Créer l'axe inset pour la minimap
        inset_ax = inset_axes(ax_main, width=config['size'], height=config['size'],
                             loc=config['position'], borderpad=0.5)
        inset_ax.patch.set_alpha(config['transparency'])

        # Créer le terrain
        field = FootballField2D(line_color='white', background_color='black', line_width=1.5)
        field.draw(ax=inset_ax, show_plot=False,
                  rotation=config['rotation'], half_field=config['half_field'],
                  invert_x=config['invert_x'], invert_y=config['invert_y'])

        # Dessiner les objets sur le terrain
        self._draw_field_objects(inset_ax, field, field_points)

        # Configuration finale
        visible_count = len(field_points)
        title = f'Frame {frame_id} - {visible_count}/{total_objects} objets projetés'
        inset_ax.set_title(title, fontsize=10, color='white', pad=8)

    def _draw_field_objects(self, ax, field: FootballField2D, field_points: Dict):
        """Dessine tous les objets sur le terrain"""
        for obj_id, point_data in field_points.items():
            # Appliquer la rotation aux coordonnées
            original_coords = np.array([[point_data['x'], point_data['y']]])
            rotated_coords = field._rotate_coordinates(original_coords, field.rotation)
            x, y = rotated_coords[0]

            # Dessiner l'objet
            self.field_drawer.draw_object(ax, x, y, point_data['info'], obj_id, point_size=150)

    def visualize_frame_to_file(self, frame_id: str, output_path: str, show_minimap: bool = True,
                               figsize: Tuple[int, int] = (15, 8), dpi: int = 100) -> bool:
        """
        Visualise une frame et la sauvegarde dans un fichier

        Args:
            frame_id: ID de la frame à traiter
            output_path: Chemin de sortie pour l'image
            show_minimap: Afficher la minimap ou non
            figsize: Taille de la figure
            dpi: Résolution de l'image de sortie

        Returns:
            bool: True si succès, False sinon
        """
        try:
            # Charger les données
            project = self.load_project_data()
            objects_config = project.get('objects', {})
            frame_annotations = project['annotations'][frame_id]
            img = self.load_frame_image(frame_id)

            # Extraire les données
            x_coords, y_coords, object_ids, object_infos = self.data_extractor.extract_image_points(
                frame_annotations, objects_config)
            field_points = self.data_extractor.extract_field_points(frame_annotations, objects_config)

            # Créer la figure principale
            fig, ax_main = plt.subplots(figsize=figsize, dpi=dpi)
            ax_main.imshow(img)
            ax_main.axis('off')

            # Dessiner les objets sur l'image
            self._draw_image_objects(ax_main, x_coords, y_coords, object_ids, object_infos)

            # Ajouter la minimap si demandée et si des points terrain sont disponibles
            if show_minimap and field_points:
                self._create_minimap(ax_main, field_points, frame_id, len(frame_annotations))

            # Sauvegarder la figure
            plt.tight_layout()
            plt.savefig(output_path, bbox_inches='tight', pad_inches=0, dpi=dpi)
            plt.close(fig)  # Fermer la figure pour libérer la mémoire

            return True

        except Exception as e:
            print(f"❌ Erreur lors du traitement de la frame {frame_id}: {e}")
            return False

    def export_video(self, output_video_path: str = None, fps: int = 30, show_minimap: bool = True,
                    figsize: Tuple[int, int] = (15, 8), dpi: int = 100, cleanup_frames: bool = True,
                    force_regenerate: bool = False) -> bool:
        """
        Exporte toutes les frames en vidéo avec annotations

        Args:
            output_video_path: Chemin de sortie pour la vidéo (optionnel)
            fps: Frames par seconde de la vidéo
            show_minimap: Afficher la minimap sur chaque frame
            figsize: Taille des figures
            dpi: Résolution des images
            cleanup_frames: Supprimer les frames temporaires après création de la vidéo
            force_regenerate: Forcer la régénération des frames même si elles existent déjà

        Returns:
            bool: True si succès, False sinon
        """
        # Définir le chemin de sortie par défaut
        if output_video_path is None:
            output_video_path = str(self.cfg.output_video_path)


        # Créer le dossier pour les frames temporaires
        temp_frames_dir = self.cfg.output_dir / "temp_annotated_frames"
        temp_frames_dir.mkdir(exist_ok=True)

        print(f"🎬 Démarrage de l'export vidéo...")
        print(f"📁 Frames temporaires: {temp_frames_dir}")
        print(f"🎥 Vidéo de sortie: {output_video_path}")

        try:
            # Charger le projet et obtenir toutes les frames
            project = self.load_project_data()
            available_frames = list(project['annotations'].keys())
            available_frames = [f for f in available_frames if f.isdigit()]
            available_frames.sort(key=int)  # Trier numériquement

            if not available_frames:
                print("❌ Aucune frame disponible pour l'export")
                return False

            print(f"📊 {len(available_frames)} frames à traiter")

            # Vérifier si les frames annotées existent déjà
            existing_frames = []
            missing_frames = []

            for frame_id in available_frames:
                frame_path = temp_frames_dir / f"frame_{int(frame_id):05d}.png"
                if frame_path.exists():
                    existing_frames.append(frame_id)
                else:
                    missing_frames.append(frame_id)

            # Décider quelles frames générer
            if not force_regenerate and existing_frames:
                print(f"📋 Frames existantes trouvées: {len(existing_frames)}")
                if missing_frames:
                    print(f"🔄 Frames manquantes à générer: {len(missing_frames)}")
                    frames_to_generate = missing_frames
                else:
                    print("✅ Toutes les frames annotées existent déjà, passage direct à la création vidéo")
                    frames_to_generate = []
            else:
                if force_regenerate:
                    print("🔄 Régénération forcée de toutes les frames")
                frames_to_generate = available_frames

            # Générer les frames nécessaires
            successful_frames = len(existing_frames) if not force_regenerate else 0
            failed_frames = []

            if frames_to_generate:
                for frame_id in tqdm(frames_to_generate, desc="🖼️  Génération des frames"):
                    output_frame_path = temp_frames_dir / f"frame_{int(frame_id):05d}.png"

                    if self.visualize_frame_to_file(frame_id, str(output_frame_path), show_minimap, figsize, dpi):
                        successful_frames += 1
                    else:
                        failed_frames.append(frame_id)

            print(f"✅ Frames totales disponibles: {successful_frames}/{len(available_frames)}")

            if failed_frames:
                print(f"⚠️  Frames échouées: {failed_frames}")

            if successful_frames == 0:
                print("❌ Aucune frame n'a pu être générée")
                return False

            # Créer la vidéo à partir des frames
            success = self._create_video_from_frames(str(temp_frames_dir), output_video_path, fps)

            # Nettoyer les frames temporaires si demandé
            if cleanup_frames and success:
                import shutil
                shutil.rmtree(temp_frames_dir)
                print("🧹 Frames temporaires supprimées")
            elif not cleanup_frames:
                print(f"💾 Frames conservées dans: {temp_frames_dir}")

            if success:
                print(f"🎉 Export vidéo terminé avec succès!")
                print(f"📹 Vidéo sauvegardée: {output_video_path}")

            return success

        except Exception as e:
            print(f"❌ Erreur lors de l'export vidéo: {e}")
            return False

    def _create_video_from_frames(self, frames_dir: str, output_video_path: str, fps: int = 30) -> bool:
        """
        Crée une vidéo à partir des images annotées

        Args:
            frames_dir: Dossier contenant les frames
            output_video_path: Chemin de sortie pour la vidéo
            fps: Frames par seconde

        Returns:
            bool: True si succès, False sinon
        """
        try:
            # Lister toutes les images
            image_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])

            if not image_files:
                print("❌ Aucune image trouvée dans le dossier")
                return False

            # Lire la première image pour obtenir les dimensions
            first_image_path = os.path.join(frames_dir, image_files[0])
            first_image = cv2.imread(first_image_path)

            if first_image is None:
                print(f"❌ Impossible de lire l'image: {first_image_path}")
                return False

            height, width, layers = first_image.shape

            # Créer le writer vidéo
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

            if not video_writer.isOpened():
                print("❌ Impossible de créer le writer vidéo")
                return False

            print(f"🎬 Création de la vidéo avec {len(image_files)} frames...")

            # Ajouter chaque image à la vidéo
            for image_file in tqdm(image_files, desc="🎞️  Assemblage vidéo"):
                image_path = os.path.join(frames_dir, image_file)
                frame = cv2.imread(image_path)

                if frame is not None:
                    video_writer.write(frame)
                else:
                    print(f"⚠️  Impossible de lire: {image_file}")

            # Finaliser la vidéo
            video_writer.release()
            return True

        except Exception as e:
            print(f"❌ Erreur lors de la création de la vidéo: {e}")
            return False

    def get_export_stats(self) -> Dict:
        """
        Retourne des statistiques sur les données disponibles pour l'export

        Returns:
            Dict: Statistiques (nombre de frames, objets, etc.)
        """
        try:
            project = self.load_project_data()
            available_frames = list(project['annotations'].keys())
            available_frames = [f for f in available_frames if f.isdigit()]

            # Compter les objets par frame
            objects_per_frame = {}
            total_objects = 0

            for frame_id in available_frames:
                frame_annotations = project['annotations'][frame_id]
                objects_count = len([ann for ann in frame_annotations if ann is not None])
                objects_per_frame[frame_id] = objects_count
                total_objects += objects_count

            stats = {
                'total_frames': len(available_frames),
                'total_objects': total_objects,
                'avg_objects_per_frame': total_objects / len(available_frames) if available_frames else 0,
                'objects_config': len(project.get('objects', {})),
                'frames_range': f"{min(available_frames)}-{max(available_frames)}" if available_frames else "N/A"
            }

            return stats

        except Exception as e:
            print(f"❌ Erreur lors du calcul des statistiques: {e}")
            return {}


In [ ]:
# =============================================================================
# 🚀 UTILISATION AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Créer le visualiseur avec la nouvelle configuration
visualizer = FootballVisualizer(viz_cfg)

# Vérifier que le fichier projet existe
if not viz_cfg.output_json_path.exists():
    print(f"❌ ERREUR: Fichier projet non trouvé: {viz_cfg.output_json_path}")
    if viz_cfg.USING_COLAB:
        print(f"💡 Vérifiez que le backup contient bien ce fichier")
    else:
        print(f"💡 Exécutez d'abord SAM_inference_refactor.ipynb pour générer ce fichier")
else:
    print(f"✅ Fichier projet trouvé: {viz_cfg.output_json_path}")

# Obtenir des statistiques
stats = visualizer.get_export_stats()
if stats:
    print(f"\n📊 STATISTIQUES DU PROJET:")
    print(f"   🎬 Frames disponibles: {stats['total_frames']}")
    print(f"   🎯 Total annotations: {stats['total_objects']}")
    print(f"   📊 Moyenne par frame: {stats['avg_objects_per_frame']:.1f}")
    print(f"   🆔 Objets configurés: {stats['objects_config']}")
    print(f"   📍 Plage de frames: {stats['frames_range']}")

# Configurer la minimap (optionnel)
visualizer.configure_minimap(
    rotation=90,
    half_field='left',
    invert_y=True,
    transparency=0.65
)

print(f"\n🎨 Visualiseur configuré et prêt!")
print(f"💡 Utilisez visualizer.visualize_frame('0') pour voir une frame")
print(f"💡 Utilisez visualizer.export_video() pour exporter la vidéo complète")

In [ ]:
visualizer.visualize_frame('0')

In [ ]:
# =============================================================================
# 🎬 EXPORT VIDÉO AVEC SAUVEGARDE INTELLIGENTE
# =============================================================================

def export_and_backup(visualizer, viz_cfg):
    """Export vidéo avec sauvegarde automatique selon l'environnement"""
    
    print(f"🎬 DÉMARRAGE DE L'EXPORT VIDÉO...")
    
    # Export de la vidéo
    success = visualizer.export_video(
        fps=3,                    # FPS de la vidéo
        show_minimap=True,        # Inclure la minimap
        dpi=100,                  # Qualité
        cleanup_frames=True,      # Nettoyer les frames temporaires
        force_regenerate=False    # Ne pas forcer si frames existent
    )
    
    if not success:
        print(f"❌ Erreur lors de l'export vidéo")
        return False
    
    print(f"✅ Export vidéo terminé!")
    print(f"📹 Vidéo: {viz_cfg.output_video_path}")
    
    # Sauvegarde selon l'environnement
    if viz_cfg.USING_COLAB:
        print(f"\n☁️ Sauvegarde Google Drive...")
        
        try:
            from google.colab import drive
            import shutil
            
            # Le Drive est déjà monté
            zip_name = viz_cfg.VIDEO_NAME
            drive_zip_path = f'/content/drive/MyDrive/{zip_name}'
            
            print(f"   📦 Mise à jour du backup: {zip_name}.zip")
            
            # Créer le ZIP mis à jour (écrase l'ancien)
            shutil.make_archive(drive_zip_path, 'zip', str(viz_cfg.videos_dir))
            
            # Vérifier la taille
            zip_path = Path(f"{drive_zip_path}.zip")
            if zip_path.exists():
                zip_size = zip_path.stat().st_size / 1024**2
                print(f"   ✅ Backup mis à jour: {zip_name}.zip ({zip_size:.1f}MB)")
                print(f"   📁 Emplacement: MyDrive/{zip_name}.zip")
            
            print(f"✅ Sauvegarde Google Drive terminée!")
            
        except Exception as e:
            print(f"❌ Erreur sauvegarde Google Drive: {e}")
    
    else:
        print(f"\n🖥️ Mode local: vidéo sauvée localement")
        print(f"📁 Dossier de sortie: {viz_cfg.output_dir}")
    
    return True

# Exécution
if viz_cfg.output_json_path.exists():
    success = export_and_backup(visualizer, viz_cfg)
    
    if success:
        print(f"\n🎉 Pipeline de visualisation terminé avec succès!")
        if viz_cfg.USING_COLAB:
            print(f"☁️ Backup mis à jour sur Google Drive: MyDrive/{viz_cfg.VIDEO_NAME}.zip")
        else:
            print(f"📁 Résultats disponibles dans: {viz_cfg.output_dir}")
    else:
        print(f"\n❌ Erreur durant le pipeline")
else:
    print(f"❌ Fichier projet manquant, impossible de continuer")

In [21]:
# =============================================================================
# ☁️ SAUVEGARDE FINALE (COLAB UNIQUEMENT)
# =============================================================================

if viz_cfg.USING_COLAB:
    print("☁️ Sauvegarde finale sur Google Drive...")
    
    try:
        from google.colab import drive
        import shutil
        
        # Le Drive est déjà monté depuis la configuration initiale
        # Pas besoin de le remonter
        
        # Créer le ZIP mis à jour avec tous les résultats
        zip_name = viz_cfg.VIDEO_NAME
        drive_zip_path = f'/content/drive/MyDrive/{zip_name}'
        
        print(f"📦 Création du backup final: {zip_name}.zip")
        print(f"📁 Source: {viz_cfg.videos_dir}")
        print(f"☁️ Destination: MyDrive/{zip_name}.zip")
        
        # Créer le ZIP (écrase l'ancien backup)
        shutil.make_archive(drive_zip_path, 'zip', str(viz_cfg.videos_dir))
        
        # Vérifier le résultat
        final_zip_path = f"{drive_zip_path}.zip"
        if Path(final_zip_path).exists():
            zip_size = Path(final_zip_path).stat().st_size / 1024**2
            print(f"✅ Backup final créé: {zip_size:.1f}MB")
            print(f"📁 Contient:")
            print(f"   🎬 Vidéo originale: {viz_cfg.VIDEO_NAME}.mp4")
            print(f"   🖼️  Frames extraites")
            print(f"   📄 Fichier projet JSON")
            print(f"   🎥 Vidéo annotée (si générée)")
            print(f"   🎭 Masques (si disponibles)")
        else:
            print("❌ Erreur: backup non créé")
        
        print("🎉 Sauvegarde Google Drive terminée!")
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
        print("💡 Vérifiez que Google Drive est bien monté")

else:
    print("🖥️ Mode Local: sauvegarde automatique désactivée")
    print(f"📁 Tous les fichiers sont disponibles dans: {viz_cfg.output_dir}")
    print("💡 Les résultats restent disponibles localement")